# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib.request,urllib.parse,urllib.error
from scipy.stats import linregress

# Import API key
from api_key import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

644

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#build query url and set config information
url= "http://api.openweathermap.org/data/2.5/weather?q=" + city + "&appid=" + weather_api_key
units = "Imperial"
lat= []
lon= []
max_temp= []
humidity= []
cloudiness= []
wind_speed= []
country= []
date=[]
cit_list= []

In [ ]:
#set counts for printing of records
count = 1
group = 1

#create loop for data to generate dataframe
print("------------------------------\nBegin Data Retrieval\n------------------------------")
for city in cities:
    print(f"Processing record {count}| {city}")
    try:
        response = requests.get(url + city.replace(" ","&")).json()
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        
        if count > 49:
            count = 1
            group + 1
            cit_list.append(city)
        else:
            count + 1
            cit_list.append(city)
    except Exception:
        print("City not found. Skipping...")
print("------------------------------\nRetrieval of Data: Completed\n------------------------------")

------------------------------
Begin Data Retrieval
------------------------------
Processing record 1| punta arenas
City not found. Skipping...
Processing record 1| padang
City not found. Skipping...
Processing record 1| severo-kurilsk
City not found. Skipping...
Processing record 1| kushiro
City not found. Skipping...
Processing record 1| alofi
City not found. Skipping...
Processing record 1| jamestown
City not found. Skipping...
Processing record 1| wanaka
City not found. Skipping...
Processing record 1| busselton
City not found. Skipping...
Processing record 1| price
City not found. Skipping...
Processing record 1| mataura
City not found. Skipping...
Processing record 1| taolanaro
City not found. Skipping...
Processing record 1| yellowknife
City not found. Skipping...
Processing record 1| outlook
City not found. Skipping...
Processing record 1| cariamanga
City not found. Skipping...
Processing record 1| tomatlan
City not found. Skipping...
Processing record 1| rikitea
City not foun

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [14]:
#Create dictionary for dataframe column creation
weather_stats = {"City": cit_list,
                 "Lat": lat,
                 "Lon": lon,
                 "Max Temp" : max_temp,
                 "Humidity" : humidity,
                 "Cloudiness" : cloudiness,
                 "Wind Speed" : wind_speed,
                 "Country" : country,
                 "Date" : date}

In [15]:
#Create a DataFrame from previously obtained data presented through dictionary
weather_data = pd.DataFrame(weather_stats)
weather_data.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression